# Temperature-Controlled Reasoning in TensorLogic

Temperature is TensorLogic's mechanism for controlling the reasoning mode:

- **T=0**: Deductive (exact boolean logic, no hallucinations)
- **T>0**: Analogical (soft probabilities, allows generalization)

## What You'll Learn

1. How temperature affects reasoning
2. Deductive vs analogical mode
3. Practical examples with uncertain knowledge
4. When to use each mode

In [ ]:
from tensorlogic import (
    create_backend,
    reason,
    quantify,
    exists,
    forall,
    temperature_scaled_operation,
    deductive_operation,
    analogical_operation,
)
import numpy as np

backend = create_backend()
print(f"Backend: {type(backend).__name__}")

## The Temperature Dial

Temperature controls the blend between hard and soft semantics:

```
T=0.0  |--------------------------|  T=∞
Exact  |     Soft Blend           |  Pure Soft
Logic  |                          |  Probabilities
```

The blending formula uses `alpha = 1 - exp(-temperature)`:
- T=0.0 → alpha=0 (100% hard)
- T=1.0 → alpha≈0.63 (63% soft)
- T=2.0 → alpha≈0.86 (86% soft)
- T→∞ → alpha=1 (100% soft)

In [ ]:
# Visualize the alpha curve
temperatures = np.linspace(0, 5, 50)
alphas = 1 - np.exp(-temperatures)

print("Temperature to Alpha Mapping:")
print("=" * 50)
print(f"{'Temperature':>12} {'Alpha':>10} {'Interpretation':>25}")
print("-" * 50)

key_temps = [0.0, 0.5, 1.0, 2.0, 5.0]
for t in key_temps:
    alpha = 1 - np.exp(-t)
    if t == 0.0:
        interp = "100% Hard (Deductive)"
    elif t < 1.0:
        interp = f"{alpha*100:.0f}% Soft"
    elif t == 1.0:
        interp = "63% Soft (Standard)"
    else:
        interp = f"{alpha*100:.0f}% Soft (Analogical)"
    print(f"{t:>12.1f} {alpha:>10.3f} {interp:>25}")

## Deductive Mode (T=0)

At temperature 0, TensorLogic performs exact boolean logic:

- EXISTS: `step(sum(x)) = 1 if any x > 0`
- FORALL: `step(prod(x)) = 1 if all x > threshold`

**Use when**: You need exact, interpretable answers with no ambiguity.

In [ ]:
# Example: Do we have any positive evidence?
evidence = np.array([0.0, 0.3, 0.0, 0.8, 0.0])  # Sparse evidence

print("Deductive Mode (T=0): Exact Boolean Logic")
print("=" * 50)
print(f"Evidence: {evidence}")

# Deductive: ANY positive value → True
result_deductive = reason(
    "exists x: P(x)",
    predicates={"P": evidence},
    bindings={"x": np.arange(len(evidence))},
    temperature=0.0,
    backend=backend,
)

result_val = float(result_deductive) if np.ndim(result_deductive) == 0 else float(result_deductive.flatten()[0])
print(f"\nQuery: exists x: P(x) (Is there any evidence?)")
print(f"Result: {result_val:.1f}")
print(f"Interpretation: {'Yes, evidence exists' if result_val > 0 else 'No evidence'}")

## Analogical Mode (T>0)

At higher temperatures, reasoning becomes softer and allows for generalization:

- EXISTS: Smooth aggregation of evidence
- FORALL: Soft product with tolerance for uncertainty

**Use when**: You have uncertain knowledge and want probabilistic reasoning.

In [ ]:
# Compare different temperatures on the same query
uncertain_evidence = np.array([0.2, 0.4, 0.6, 0.3, 0.1])  # All uncertain

print("Temperature Comparison: exists x: P(x)")
print("=" * 60)
print(f"Evidence: {uncertain_evidence}")
print("\n" + "-" * 60)
print(f"{'Temperature':>12} {'Result':>10} {'Interpretation'}")
print("-" * 60)

for temp in [0.0, 0.5, 1.0, 2.0, 5.0]:
    result = reason(
        "exists x: P(x)",
        predicates={"P": uncertain_evidence},
        bindings={"x": np.arange(len(uncertain_evidence))},
        temperature=temp,
        backend=backend,
    )
    result_val = float(result) if np.ndim(result) == 0 else float(result.flatten()[0])
    
    if temp == 0.0:
        interp = "Hard: step(sum) = 1 if any > 0"
    else:
        alpha = 1 - np.exp(-temp)
        interp = f"Soft blend: {(1-alpha)*100:.0f}% hard + {alpha*100:.0f}% soft"
    
    print(f"{temp:>12.1f} {result_val:>10.3f} {interp}")

## Practical Example: Uncertain Parentage

Consider a knowledge graph where we have uncertain information about family relationships.

In [ ]:
# Uncertain "possible parent" relation with confidence values
entities = ["Alice", "Bob", "Carol", "David", "Eve"]
n = len(entities)
entity_to_idx = {name: i for i, name in enumerate(entities)}

possible_parent = np.zeros((n, n), dtype=np.float32)
# Uncertain facts with confidence levels
possible_parent[0, 2] = 0.9   # Alice possibly parent of Carol (90%)
possible_parent[0, 3] = 0.4   # Alice possibly parent of David (40% - actually not)
possible_parent[1, 2] = 0.85  # Bob possibly parent of Carol (85%)
possible_parent[2, 4] = 0.75  # Carol possibly parent of Eve (75%)

print("Uncertain Knowledge: PossibleParent Relation")
print("=" * 50)
for i, p_name in enumerate(entities):
    for j, c_name in enumerate(entities):
        if possible_parent[i, j] > 0:
            print(f"  {p_name} -> {c_name}: {possible_parent[i, j]:.0%} confident")

In [ ]:
# Query: Does Alice have any children?
alice_row = possible_parent[0, :]  # Alice's row

print("\nQuery: Does Alice possibly have children?")
print("=" * 50)
print(f"Alice's row: {alice_row}")
print(f"(Non-zero: Carol@90%, David@40%)")
print("\n" + "-" * 50)

for temp in [0.0, 0.5, 1.0, 2.0]:
    result = reason(
        "exists y: P(y)",
        predicates={"P": alice_row},
        bindings={"y": np.arange(n)},
        temperature=temp,
        backend=backend,
    )
    result_val = float(result) if np.ndim(result) == 0 else float(result.flatten()[0])
    
    print(f"\nT={temp:.1f}: Result = {result_val:.3f}")
    if temp == 0.0:
        print("  Interpretation: Definite yes (any evidence exists)")
    else:
        print(f"  Interpretation: {result_val:.0%} confident Alice has children")

## Implication with Temperature

How does temperature affect implication evaluation?

In [ ]:
# Implication: Parent(x,y) -> Loves(x,y)
# Does Alice (who is only 40% sure she's David's parent) love David?

parent_confidence = np.array([0.4])  # 40% confident Alice is David's parent
loves_value = np.array([0.0])        # Alice doesn't love David (based on our knowledge)

print("Implication with Uncertainty")
print("=" * 60)
print(f"Fact: Alice possibly parent of David with {float(parent_confidence[0]):.0%} confidence")
print(f"Fact: Alice loves David with {float(loves_value[0]):.0%} confidence")
print("\nQuery: Does Parent(Alice, David) -> Loves(Alice, David) hold?")
print("-" * 60)

# Implication: max(1-P, Q) = max(1-0.4, 0.0) = 0.6 (soft)
print("\nClassical implication: max(1-P, Q) = max(1-0.4, 0.0) = 0.6")
print("(Implication holds when P is weak or Q is strong)")

for temp in [0.0, 1.0, 2.0]:
    result = reason(
        "P(x) -> Q(x)",
        predicates={"P": parent_confidence, "Q": loves_value},
        bindings={"x": np.array([0])},
        temperature=temp,
        backend=backend,
    )
    result_val = float(result) if np.ndim(result) == 0 else float(result.flatten()[0])
    
    print(f"\nT={temp:.1f}: Implication = {result_val:.3f}")
    if temp == 0.0:
        print("  Hard: step(0.6) = 1.0 (technically holds because P is weak)")
    else:
        print(f"  Soft: Blended result accounting for uncertainty")

## When to Use Each Mode

| Temperature | Mode | Use Case |
|-------------|------|----------|
| T=0 | Deductive | Database queries, constraint checking, rule validation |
| T=0.5-1.0 | Balanced | General reasoning with some uncertainty |
| T=2.0+ | Analogical | Soft similarity matching, fuzzy reasoning, exploration |

In [ ]:
# Use Case Examples

print("Use Case Examples")
print("=" * 70)

# Use Case 1: Database Query (Deductive)
print("\n1. DATABASE QUERY (T=0, Deductive)")
print("-" * 50)
print("Question: Is there a user with admin privileges?")
admin_flags = np.array([0, 0, 1, 0, 0], dtype=np.float32)  # User 2 is admin
result = reason(
    "exists x: Admin(x)",
    predicates={"Admin": admin_flags},
    bindings={"x": np.arange(5)},
    temperature=0.0,
    backend=backend,
)
result_val = float(result) if np.ndim(result) == 0 else float(result.flatten()[0])
print(f"Answer: {'Yes' if result_val > 0 else 'No'} (exact boolean)")

# Use Case 2: Recommendation (Balanced)
print("\n2. RECOMMENDATION SYSTEM (T=1.0, Balanced)")
print("-" * 50)
print("Question: How strongly do user preferences match item features?")
user_prefs = np.array([0.8, 0.6, 0.3])
item_features = np.array([0.7, 0.5, 0.2])
# Soft AND of all feature matches
match_score = np.mean(user_prefs * item_features)  # Simple soft matching
print(f"Match score: {match_score:.2f} (soft probability)")

# Use Case 3: Exploration (Analogical)
print("\n3. KNOWLEDGE EXPLORATION (T=2.0, Analogical)")
print("-" * 50)
print("Question: Could this entity be related to the target?")
weak_evidence = np.array([0.1, 0.2, 0.15, 0.05])
result = reason(
    "exists x: Related(x)",
    predicates={"Related": weak_evidence},
    bindings={"x": np.arange(4)},
    temperature=2.0,
    backend=backend,
)
result_val = float(result) if np.ndim(result) == 0 else float(result.flatten()[0])
print(f"Possibility score: {result_val:.3f} (soft exploration)")
print("(Higher temperature allows weak evidence to contribute)")

## Summary

**Temperature is TensorLogic's dial between exact and soft reasoning:**

- **T=0 (Deductive)**:
  - Exact boolean logic
  - No hallucinations
  - Best for: Database queries, constraint checking, rule validation

- **T>0 (Analogical)**:
  - Soft probabilistic reasoning
  - Allows generalization from uncertainty
  - Best for: Recommendations, exploration, fuzzy matching

**Key Insight**: The same logical query can give different results depending on temperature, letting you choose the right reasoning mode for your application.

## Complete Tutorial Series

You've completed all four tutorials:

1. ✅ **Getting Started**: Installation, basic operations, first knowledge graph
2. ✅ **Knowledge Graphs**: Multi-hop reasoning, grandparent/aunt-uncle inference
3. ✅ **Compilation Strategies**: Different semantic interpretations
4. ✅ **Temperature Control**: Deductive vs analogical reasoning

## Next Steps

- Explore the `examples/` directory for more complex use cases
- Check `docs/research/rag-goals.md` for RAG integration roadmap
- Read the original paper: arXiv:2510.12269